# Prova 2 (2°/2019) - Solução da Prova 2
## DPEE805 - Modelagem e Controle de Conversores
### Prof° Cassiano Rech | Acadêmico: Tiarles da Rocha Moralles Guterres

#### Informações gerais sobre este arquivo:
Este arquivo é um formato impresso de um ambiente em Python chamado Jupyter-notebook. Com o Jupyter-notebook você pode escrever código e documentá-lo simultaneamente, intercalando no que chamamos de células de código, em Python, e células de texto. Apesar de utilizar o navegador e o formato ``.html`` este software possui **nenhuma conexão com a internet**.

#### Um pouco sobre sintaxe:
Python, diferente de MATLAB, não existe um direcionamento específico para cálculos matriciais e modelagem. Porém, utilizando alguns repositórios (também chamados de módulos) podemos realizar quase todos os procedimentos feitos em MATLAB e de forma bem simples. Python e seus pacotes são *open source*.

Os módulos utilizados neste notebook e importados na célula abaixo são:
* **matplotlib.pyplot**: Para a geração de figuras a partir de vetores.
* **numpy**: É a biblioteca numérica de Python, onde estão implementadas as estruturas e operações com vetores/matrizes.
* **control**: Biblioteca com funções semelhantes as do MATLAB para manipular sistemas utilizando diagrama de blocos.


In [3]:
import control as ctrl
import matplotlib.pyplot as plt
import numpy as np

%matplotlib qt

**Algumas Funções**

In [4]:
def fz_eq(fc_desired, theta):
    num = 1 - np.sin(theta)
    den = 1 + np.sin(theta)

    return fc_desired * np.sqrt(num/den)


def fl_eq(fc_desired, theta):
    num = 1 + np.sin(theta)
    den = 1 - np.sin(theta)

    return fc_desired * np.sqrt(num/den)


def w_eq(f):
    return 2*np.pi*f


def deg_to_radians(deg):
    return deg/180*np.pi


def radians_to_deg(radius):
    return radius/np.pi*180

**Parâmetros do conversor**

In [25]:
L = 1e-3
rL = .1
Cref = 470e-6
R1 = 175
R2 = 350

fs = 20e3
Lf = 2.5e-3
Cdc = 1000e-6

Vm = 1
Hi = 1
Hv = 1/100
# Hca = 1/311

Vin = 220
Vo = 400

**Funções de Transferência do sistema**

In [6]:
s = ctrl.tf([1, 0], [1]); print(s)

# Gid = -Vo/(2*Lf*s)
# Gvi = np.sqrt(2) * Vin_p/(2 * Vo) * 1/(s*Cdc)


s
-
1



## Análise da Malha de corrente (Malha Rápida)

**Compensador utilizado:** PI. Por necessitar de margem de fase numa frequência de cruzamento específica com adição de um pólo para compensar o efeito de comutação existente que a planta simplificada desconsiderou

In [11]:
import warnings
warnings.filterwarnings('ignore')

# T = Gid * Hi/Vm

Ti = -400/(2*2.5e-3*s)

In [19]:
fc = fs/2
theta = deg_to_radians(60)

fz = fz_eq(fc, theta); print('fz ', fz)
wz = w_eq(fz); print('wz ', wz)

fz  2679.4919243112276
wz  16835.744289538663


In [20]:
Ci = -(s + wz) / s

In [21]:
ctrl.sisotool(Ci * Ti, Hz=True, dB=True, omega_num=1000, omega_limits=(1, 1e6))

Clicked at      -3159+1.016e+04j gain   -0.08129 damp     0.2968
Clicked at      -3159+1.016e+04j gain   -0.08129 damp     0.2968
Clicked at      -3159+1.016e+04j gain    0.08129 damp     0.2968
Clicked at      -3159+1.016e+04j gain    0.08304 damp     0.2968
Clicked at      -9017+1.448e+04j gain    -0.1581 damp     0.5285
Clicked at      -9017+1.448e+04j gain    -0.1581 damp     0.5285
Clicked at      -9017+1.448e+04j gain     0.1581 damp     0.5285
Clicked at      -9017+1.448e+04j gain     0.2209 damp     0.5285
Clicked at      -1903     +6804j gain    -0.0616 damp     0.2694
Clicked at      -1903     +6804j gain    -0.0616 damp     0.2694
Clicked at      -1903     +6804j gain     0.0616 damp     0.2694
Clicked at      -1903     +6804j gain    0.03776 damp     0.2694
Clicked at      -3995+1.016e+04j gain   -0.09289 damp     0.3659
Clicked at      -3995+1.016e+04j gain   -0.09289 damp     0.3659
Clicked at      -3995+1.016e+04j gain    0.09289 damp     0.3659
Clicked at      -3995+1.0

**Diagrama de bode após o ajuste no "sisotool"**

In [23]:
Ci_ajus = 0.1 * Ci
plt.clf()
_,_,_ = ctrl.bode(Ci_ajus * Ti, Hz=True, dB=True, margins=True, omega_num=10000, omega_limits=(1, 1e6))#, Plot=False)

## Análise da Malha de tensão (Malha Lenta)

**Compensador utilizado:** PI

$C_{PI} = K_{PI} \frac{1}{s}$

In [32]:
Kv = 1 /np.sqrt(2) # Senóide de amplitude 1 V 

T2 = 220/(2*400) * 1/(s*1000e-6) * 1/100

In [36]:
Cv = (s + 2)/s  # Planta com pólo nulo

In [37]:
plt.clf()
ctrl.sisotool(Cv * T2, Hz=True, dB=True)

Clicked at     -12.16  +0.08598j gain      5.291 damp          1
Clicked at     -19.26    +0.181j gain      7.817 damp          1
Clicked at     -1.767    +1.968j gain      1.284 damp     0.6682
Clicked at     -3.507    +1.207j gain       2.59 damp     0.9455
Clicked at      329.7    -5.539j gain     -119.2 damp    -0.9999
Clicked at     -282.9    +1.258j gain      103.6 damp          1
Clicked at     -146.1   +0.2509j gain      53.87 damp          1
Clicked at     -61.01   +0.2509j gain      22.94 damp          1
Clicked at     -45.66   -0.2526j gain      17.36 damp          1
Clicked at     -38.68-0.0008615j gain      14.83 damp          1


**Diagrama de bode após ajustar o ganho no "sisotool"**

In [38]:
Cv_adjus = 14*Cv
plt.clf()
_,_,_ = ctrl.bode(Cv_adjus * T2, Hz=True, dB=True, margins=True, omega_num=10000, omega_limits=(1, 1e6))#, Plot=False)